In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['data']


In [2]:
import numpy as np
import gzip
import copy
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset
from torch.utils.data import SubsetRandomSampler, DataLoader
import os


class_mapping = {
    'Rock': 'Rock',
    'Psych-Rock': 'Rock',
    'Indie-Rock': None,
    'Post-Rock': 'Rock',
    'Psych-Folk': 'Folk',
    'Folk': 'Folk',
    'Metal': 'Metal',
    'Punk': 'Metal',
    'Post-Punk': None,
    'Trip-Hop': 'Trip-Hop',
    'Pop': 'Pop',
    'Electronic': 'Electronic',
    'Hip-Hop': 'Hip-Hop',
    'Classical': 'Classical',
    'Blues': 'Blues',
    'Chiptune': 'Electronic',
    'Jazz': 'Jazz',
    'Soundtrack': None,
    'International': None,
    'Old-Time': None
}


def torch_train_val_split(
        dataset, batch_train, batch_eval,
        val_size=.2, shuffle=True, seed=42):
    # Creating data indices for training and validation splits:
    dataset_size = len(dataset)
    indices = list(range(dataset_size))
    val_split = int(np.floor(val_size * dataset_size))
    if shuffle:
        np.random.seed(seed)
        np.random.shuffle(indices)
    train_indices = indices[val_split:]
    val_indices = indices[:val_split]

    # Creating PT data samplers and loaders:
    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)

    train_loader = DataLoader(dataset,
                              batch_size=batch_train,
                              sampler=train_sampler)
    val_loader = DataLoader(dataset,
                            batch_size=batch_eval,
                            sampler=val_sampler)
    return train_loader, val_loader


def read_spectrogram(spectrogram_file, chroma=True):
    with gzip.GzipFile(spectrogram_file, 'r') as f:
        spectrograms = np.load(f)
    # spectrograms contains a fused mel spectrogram and chromagram
    # Decompose as follows
    return spectrograms.T


class LabelTransformer(LabelEncoder):
    def inverse(self, y):
        try:
            return super(LabelTransformer, self).inverse_transform(y)
        except:
            return super(LabelTransformer, self).inverse_transform([y])

    def transform(self, y):
        try:
            return super(LabelTransformer, self).transform(y)
        except:
            return super(LabelTransformer, self).transform([y])

        
class PaddingTransform(object):
    def __init__(self, max_length, padding_value=0):
        self.max_length = max_length
        self.padding_value = padding_value

    def __call__(self, s):
        if len(s) == self.max_length:
            return s

        if len(s) > self.max_length:
            return s[:self.max_length]

        if len(s) < self.max_length:
            s1 = copy.deepcopy(s)
            pad = np.zeros((self.max_length - s.shape[0], s.shape[1]), dtype=np.float32)
            s1 = np.vstack((s1, pad))
            return s1

        
class SpectrogramDataset(Dataset):
    def __init__(self, path, class_mapping=None, train=True, max_length=-1):
        t = 'train' if train else 'test'
        p = os.path.join(path, t)
        self.index = os.path.join(path, "{}_labels.txt".format(t))
        self.files, labels = self.get_files_labels(self.index, class_mapping)
        #print(self.files)
        
        self.feats = [read_spectrogram(os.path.join(p, f+".fused.full.npy.gz")) for f in self.files]
        self.feat_dim = self.feats[0].shape[1]
        self.lengths = [len(i) for i in self.feats]
        self.max_length = max(self.lengths) if max_length <= 0 else max_length
        self.zero_pad_and_stack = PaddingTransform(self.max_length)
        #self.label_transformer = LabelTransformer()
        #if isinstance(labels, (list, tuple)):
            #self.labels = np.array(self.label_transformer.fit_transform(labels)).astype('int64')
        self.labels=labels
    def get_files_labels(self, txt, class_mapping):
        with open(txt, 'r') as fd:
            lines = [l.rstrip().split('\t') for l in fd.readlines()[1:]]
            
        files, labels = [], []
        for l in lines:
            l=l[0].split(",")
            b=l[1:]
            b = list(map(float,b))
            files.append(l[0])
            
            labels.append(b)
        return files, labels

    def __getitem__(self, item):
        l = min(self.lengths[item], self.max_length)
        return self.zero_pad_and_stack(self.feats[item]), self.labels[item], l

    def __len__(self):
        return len(self.labels)

In [3]:
BATCH_SZ=32

specs = SpectrogramDataset('../input/data/data/multitask_dataset/', train=True, class_mapping=class_mapping, max_length=-1)
train_loader, val_loader = torch_train_val_split(specs, BATCH_SZ ,BATCH_SZ, val_size=0)

In [4]:
import numpy as np
import torch
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from scipy import stats

class ConvNetMulty3(nn.Module):
    def __init__(self,input_channels,out_channels,kernel_sz,stride,padding, num_classes):
        super(ConvNetMulty3, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(input_channels, 16, kernel_size=(5,5), stride=1, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=(5,5), stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(32 , 64 , kernel_size=(3,3), stride=1, padding=1),
            
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer4_task1 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=(3,3), stride=2, padding=1),
            
            nn.BatchNorm2d(128),
            nn.ReLU(),
            
            nn.MaxPool2d(kernel_size=3, stride=3)
        )
        self.layer4_task2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=(3,3), stride=2, padding=1),
            
            nn.BatchNorm2d(128),
            nn.ReLU(),
            
            nn.MaxPool2d(kernel_size=3, stride=3)
        )
        self.layer4_task3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=(3,3), stride=2, padding=1),
            
            nn.BatchNorm2d(128),
            nn.ReLU(),
           
            nn.MaxPool2d(kernel_size=3, stride=3)
        )
        
        #self.dense1_task1= nn.Linear(6720,500) 
        #self.dense2_task1 = nn.Linear(500,50)
        #self.dense3_task1 = nn.Linear(50,1)
        self.dense_task1 = nn.Sequential(
            nn.Linear(6656,1000),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1000,500),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(500,50),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(50,1),
            
        ) 
        
        #self.dense1_task2= nn.Linear(6720,500) 
        #self.dense2_task2 = nn.Linear(500,50)
        #self.dense3_task2 = nn.Linear(50,1)
        self.dense_task2 = nn.Sequential(
            nn.Linear(6656,1000),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1000,500),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(500,50),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(50,1)
        ) 
        
        #self.dense1_task3= nn.Linear(6720,500) 
        #self.dense2_task3 = nn.Linear(500,50)
        #self.dense3_task3 = nn.Linear(50,1)
        self.dense_task3 = nn.Sequential(
            nn.Linear(6656,1000),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1000,500),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(500,50),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(50,1)
        ) 
        
    def forward(self, x,lengths):
        #print(x.shape)
        x = x.transpose(1, 2)
        #print(x.shape)
        x.unsqueeze_(1)
        #print(x.shape)
        out1 = self.layer1(x)
        #print(out1.shape)
        out2= self.layer2(out1)
        #print(out2.shape)
        out3= self.layer3(out2)
        #print(out3.shape)
        #out4= self.layer4(out3)
        
        out4_task1= self.layer4_task1(out3)
        out4_task2= self.layer4_task2(out3)
        out4_task3= self.layer4_task3(out3)
        
        #print(out4.shape)
        out_flat_task1=out4_task1.reshape(-1,out4_task1.size(1)*out4_task1.size(2)*out4_task1.size(3))
        out_flat_task2=out4_task2.reshape(-1,out4_task2.size(1)*out4_task2.size(2)*out4_task2.size(3))
        out_flat_task3=out4_task3.reshape(-1,out4_task3.size(1)*out4_task3.size(2)*out4_task3.size(3))
    
        #out_flat=out4.reshape(-1,out4.size(1)*out4.size(2)*out4.size(3))
        #print(out_flat.shape)
        
        
        #implementing fully connected layers
        
        #hidden_out_task1 = self.dense1_task1(out_flat_task1)
        #hidden_out2_task1 = self.dense2_task1(hidden_out_task1)
        #final_out_task1 =  self.dense3_task1(hidden_out2_task1)
        final_out_task1 = self.dense_task1(out_flat_task1)
        
        #hidden_out_task2 = self.dense1_task2(out_flat_task2)
        #hidden_out2_task2 = self.dense2_task2(hidden_out_task2)
        #final_out_task2 = self.dense3_task2(hidden_out2_task2)
        final_out_task2 = self.dense_task2(out_flat_task2)
        
        #hidden_out_task3 = self.dense1_task3(out_flat_task3)
        #hidden_out2_task3 = self.dense2_task3(hidden_out_task3)
        #final_out_task3 = self.dense3_task3(hidden_out2_task3)
        final_out_task3 = self.dense_task3(out_flat_task3)
        
        return final_out_task1,final_out_task2,final_out_task3
        

In [5]:
num_epochs=35
kernel_sz=3
input_channels=1
out_channels=1
stride=2
padding=2
num_classes=1


device=torch.device("cuda")

model_cnn_regr_multy3 = ConvNetMulty3(input_channels,out_channels,kernel_sz,stride,padding ,num_classes)
model_cnn_regr_multy3.to(device)

ConvNetMulty3(
  (layer1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer4_task1): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    

In [6]:
# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model_cnn_regr_multy3.parameters(),weight_decay=0.001)



for epoch in range(num_epochs):
    #no need to set requires_grad=True for parameters(weights) as it done by default. Also for input requires_grad is not
    #always necessary. So we comment the following line.
    #with torch.autograd(): 
    model_cnn_regr_multy3.train()
    #scheduler.step()
    running_average_loss = 0

    #train model in each epoch
    for index,instance in enumerate(train_loader):
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        #features,labels,lengths=instance
        
        features = instance[:][0].to(device)
        labels = instance[:][1]
        valence_labels = labels[0].type(torch.FloatTensor).to(device)
        energy_labels = labels[1].type(torch.FloatTensor).to(device)
        dance_labels = labels[2].type(torch.FloatTensor).to(device)
        lengths = instance[:][2].to(device)
        features = features.type(torch.FloatTensor).to(device)

        optimizer.zero_grad()
        
        # Step 3. Run our forward pass.
        prediction_vec_task1,prediction_vec_task2,prediction_vec_task3 = model_cnn_regr_multy3(features,lengths)
        prediction_vec_task1.to(device)
        prediction_vec_task2.to(device)
        prediction_vec_task3.to(device)
        
        
        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        valence_labels = valence_labels.unsqueeze(1)
        energy_labels = energy_labels.unsqueeze(1)
        dance_labels = dance_labels.unsqueeze(1)
        
        loss1 = criterion(prediction_vec_task1,valence_labels)
        loss2 = criterion(prediction_vec_task2,energy_labels)
        loss3 = criterion(prediction_vec_task3,dance_labels)
        
        
        
        loss=loss1+loss2+loss3
        
        #print(loss1,"  ",loss2,"  ",loss3)
        loss.backward(retain_graph=True)
        optimizer.step()

        running_average_loss += loss.detach().item()
    print("Epoch: {} \t \t Training Loss {}".format(epoch, float(running_average_loss) / (index + 1)),loss1.item(),loss2.item(),loss3.item())

Epoch: 0 	 	 Training Loss 1.3707121403680906 0.14447759091854095 0.0428374744951725 0.061876825988292694
Epoch: 1 	 	 Training Loss 0.2868082908292611 0.15440733730793 0.01638169400393963 0.01803939789533615
Epoch: 2 	 	 Training Loss 0.2500380728807714 0.09336184710264206 0.09637004137039185 0.005706711206585169
Epoch: 3 	 	 Training Loss 0.2449434573451678 0.0995127335190773 0.13985134661197662 0.04725595936179161
Epoch: 4 	 	 Training Loss 0.21083492247594726 0.05170074477791786 0.05883922427892685 0.02560330741107464
Epoch: 5 	 	 Training Loss 0.1998962356398503 0.05391919985413551 0.05393308401107788 0.006655864883214235
Epoch: 6 	 	 Training Loss 0.21003466058108541 0.005336370784789324 0.1293434053659439 0.09535902738571167
Epoch: 7 	 	 Training Loss 0.20595389190647337 0.11262175440788269 0.09753444790840149 0.024063611403107643
Epoch: 8 	 	 Training Loss 0.18377308340536225 0.05901624634861946 0.04387233406305313 0.0344407819211483
Epoch: 9 	 	 Training Loss 0.176094184319178

In [7]:
import numpy as np
import gzip
import copy
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset
from torch.utils.data import SubsetRandomSampler, DataLoader
import os


class_mapping = {
    'Rock': 'Rock',
    'Psych-Rock': 'Rock',
    'Indie-Rock': None,
    'Post-Rock': 'Rock',
    'Psych-Folk': 'Folk',
    'Folk': 'Folk',
    'Metal': 'Metal',
    'Punk': 'Metal',
    'Post-Punk': None,
    'Trip-Hop': 'Trip-Hop',
    'Pop': 'Pop',
    'Electronic': 'Electronic',
    'Hip-Hop': 'Hip-Hop',
    'Classical': 'Classical',
    'Blues': 'Blues',
    'Chiptune': 'Electronic',
    'Jazz': 'Jazz',
    'Soundtrack': None,
    'International': None,
    'Old-Time': None
}


def torch_train_val_split(
        dataset, batch_train, batch_eval,
        val_size=.2, shuffle=True, seed=42):
    # Creating data indices for training and validation splits:
    dataset_size = len(dataset)
    indices = list(range(dataset_size))
    val_split = int(np.floor(val_size * dataset_size))
    if shuffle:
        np.random.seed(seed)
        np.random.shuffle(indices)
    train_indices = indices[val_split:]
    val_indices = indices[:val_split]

    # Creating PT data samplers and loaders:
    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)

    train_loader = DataLoader(dataset,
                              batch_size=batch_train,
                              sampler=train_sampler)
    val_loader = DataLoader(dataset,
                            batch_size=batch_eval,
                            sampler=val_sampler)
    return train_loader, val_loader


def read_spectrogram(spectrogram_file, chroma=True):
    with gzip.GzipFile(spectrogram_file, 'r') as f:
        spectrograms = np.load(f)
    # spectrograms contains a fused mel spectrogram and chromagram
    # Decompose as follows
    return spectrograms.T


class LabelTransformer(LabelEncoder):
    def inverse(self, y):
        try:
            return super(LabelTransformer, self).inverse_transform(y)
        except:
            return super(LabelTransformer, self).inverse_transform([y])

    def transform(self, y):
        try:
            return super(LabelTransformer, self).transform(y)
        except:
            return super(LabelTransformer, self).transform([y])

        
class PaddingTransform(object):
    def __init__(self, max_length, padding_value=0):
        self.max_length = max_length
        self.padding_value = padding_value

    def __call__(self, s):
        if len(s) == self.max_length:
            return s

        if len(s) > self.max_length:
            return s[:self.max_length]

        if len(s) < self.max_length:
            s1 = copy.deepcopy(s)
            pad = np.zeros((self.max_length - s.shape[0], s.shape[1]), dtype=np.float32)
            s1 = np.vstack((s1, pad))
            return s1

        
class SpectrogramDataset_Test(Dataset):
    def __init__(self, path, class_mapping=None, train=True, max_length=-1):
        t = 'train' if train else 'test'
        p = os.path.join(path, t)
        
        #self.index = os.path.join(path, "{}_labels.txt".format(t))
        self.index = p
        self.files, labels = self.get_files_labels()
        
        self.feats = [read_spectrogram(os.path.join(p, f)) for f in self.files]
        self.feat_dim = self.feats[0].shape[1]
        self.lengths = [len(i) for i in self.feats]
        self.max_length = max(self.lengths) if max_length <= 0 else max_length
        self.zero_pad_and_stack = PaddingTransform(self.max_length)

        self.labels=labels
        
    def get_files_labels(self):
        
        """
        with open(txt, 'r') as fd:
            lines = [l.rstrip().split('\t') for l in fd.readlines()[1:]]
            
        files, labels = [], []
        for l in lines:
            l=l[0].split(",")
            b=l[1:]
            b = list(map(float,b))
            files.append(l[0])
            
            labels.append(b)
        return files, labels
        """
        files = os.listdir(self.index)
        labels=files
        return files,labels

    def __getitem__(self, item):
        l = min(self.lengths[item], self.max_length)
        return self.zero_pad_and_stack(self.feats[item]), self.labels[item], l

    def __len__(self):
        return len(self.labels)

In [8]:
test_loader = DataLoader(SpectrogramDataset_Test('../input/data/data/multitask_dataset/', train=False, class_mapping=class_mapping, max_length=-1))

In [9]:
model_cnn_regr_multy3.eval()

n_samples = 0
SE = 0
spearman1=[]
spearman2=[]
spearman3=[]
running_average_loss=0
txt="solutions_3.txt"
with open(txt, 'w') as fd:
    fd.write("Id.fused.full.npy.gz,valence,energy,danceability\n")
    with torch.no_grad():

        for index, instance in enumerate(test_loader):
            features = instance[:][0].to(device)
            label = instance[:][1]
            

            lengths = instance[:][2].to(device)
            features = features.type(torch.FloatTensor).to(device)



            out1,out2,out3 = model_cnn_regr_multy3(features,lengths)
            out1.to(device)
            out2.to(device)
            out3.to(device)
            
            fd.write(label[0]+","+str(out1.item())+","+str(out2.item())+","+str(out3.item())+"\n")

In [10]:


from subprocess import check_output
print(check_output(["ls", "./"]).decode("utf8"))



__notebook__.ipynb
__output__.json
solutions_3.txt

